# Guided Adversarial Training (GAT)

In [1]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '0,1,2,3'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/gat/%s/%s' % (model_type, dataset)
num_classes = 10
lr = 0.1
momentum = 0.9
weight_decay = 5e-4
batch_size = 128
total_epochs = 100
lam = 3.0 # ResNet-18 10.0
step_up = 20.0 # ResNet-18 4.0
epsilon = 8/255
alpha = 2/255
num_repeats = 1

## Inner maximization

In [3]:
def inner_max(model, xent, inputs, targets, epsilon, lam, num_repeats, alt):
    bern = torch.bernoulli(torch.empty(inputs.shape).uniform_(0, 1))
    delta = torch.where(bern==0, -epsilon/2, epsilon/2).cuda()
    x = torch.clamp(inputs + delta, min=0, max=1)
    epsilon = epsilon/num_repeats
    for _ in range(num_repeats):
        x.requires_grad_()
        logits_nat = model(inputs)
        logits_adv = model(x)
        prob_nat = logits_nat.softmax(dim=-1)
        prob_adv = logits_adv.softmax(dim=-1)
        loss = xent(logits_adv, targets) + alt*lam*torch.sum((prob_adv-prob_nat)**2, dim=1).mean(dim=0)
        loss.backward()
        grads = x.grad.data
        x = x.detach() + epsilon*torch.sign(grads).detach()
        #x = torch.min(torch.max(x, inputs-epsilon), inputs+epsilon).clamp(min=0, max=1)
        x = torch.clamp(x, min=0, max=1)
    return x

## Training (Outer minimization)

In [4]:
def training(epoch, model, dataloader, optimizer, num_classes, epsilon=8/255, lam=10, num_repeats=1):
    model.train()
    counter = 0
    total = 0
    total_loss = 0
    total_correct = 0
        
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        x = inner_max(model, xent, inputs, targets, epsilon, lam, num_repeats, alt=counter%2)
        logits_nat = model(inputs)
        logits_adv = model(x)
        
        prob_nat = logits_nat.softmax(dim=1)
        prob_adv = logits_adv.softmax(dim=1)
        loss = xent(logits_nat, targets) + lam*torch.sum((prob_adv-prob_nat)**2, dim=1).mean(dim=0)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
        total_correct += num_correct
        
        counter = counter + 1
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), total_loss/(idx+1),
                     num_correct/batch, total_correct/total))

In [5]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
        
            noise = torch.FloatTensor(inputs.shape).uniform_(-epsilon,epsilon).cuda()
            x = torch.clamp(inputs+noise, min=0, max=1)
            for _ in range(num_repeats):
                x.requires_grad_()
                logits = model(x)
                loss = xent(logits, targets)
                loss.backward()
                grads = x.grad.data
                x = x.detach() + alpha*torch.sign(grads).detach()
                x = torch.min(torch.max(x, inputs - epsilon), inputs + epsilon).clamp(min=0, max=1)
        
            with torch.no_grad():
                logits_nat = model(inputs)
                logits_adv = model(x)
        
            total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
            total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
            pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                         acc_rob=total_correct_adv / len(dataloader.dataset)))
        
    return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer = optim.SGD(model.parameters(),lr=lr, momentum=momentum, weight_decay=weight_decay)

scheduler = [55, 70, 75] # ResNet-18 [70, 85]
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
best_acc_nat, best_acc_rob = 0, 0

for epoch in range(total_epochs):
    training(epoch, model, train_dataloader, optimizer, num_classes, epsilon, lam, num_repeats)
    test_acc_nat, test_acc_rob = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats=10)
        
    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict': model.state_dict(),
                       'best_acc_nat': best_acc_nat,
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model'))
    adjust_learning_rate.step()
    if epoch+1 == scheduler[1]:
        lam = lam*step_up

Files already downloaded and verified
Files already downloaded and verified
Epoch 0 [0/383] | loss: 2.3218 (avg: 2.3218) | acc: 0.0625 (avg: 0.0625) |
Epoch 0 [100/383] | loss: 1.7272 (avg: 2.0287) | acc: 0.1719 (avg: 0.1407) |
Epoch 0 [200/383] | loss: 1.7385 (avg: 1.8983) | acc: 0.1797 (avg: 0.1538) |
Epoch 0 [300/383] | loss: 1.6398 (avg: 1.8149) | acc: 0.1797 (avg: 0.1655) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:43<00:00,  4.48it/s, acc_nat=0.495, acc_rob=0.161]


Epoch 1 [0/383] | loss: 1.5420 (avg: 1.5420) | acc: 0.2109 (avg: 0.2109) |
Epoch 1 [100/383] | loss: 1.5591 (avg: 1.5026) | acc: 0.1797 (avg: 0.2125) |
Epoch 1 [200/383] | loss: 1.4320 (avg: 1.4822) | acc: 0.2422 (avg: 0.2205) |
Epoch 1 [300/383] | loss: 1.2705 (avg: 1.4561) | acc: 0.3281 (avg: 0.2289) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:42<00:00,  4.49it/s, acc_nat=0.604, acc_rob=0.228]


Epoch 2 [0/383] | loss: 1.3840 (avg: 1.3840) | acc: 0.2734 (avg: 0.2734) |
Epoch 2 [100/383] | loss: 1.3693 (avg: 1.3468) | acc: 0.2344 (avg: 0.2577) |
Epoch 2 [200/383] | loss: 1.2661 (avg: 1.3257) | acc: 0.3438 (avg: 0.2641) |
Epoch 2 [300/383] | loss: 1.2594 (avg: 1.3167) | acc: 0.3516 (avg: 0.2684) |


...

[eval] 1000/1000: 100%|██████████| 1000/1000 [03:24<00:00,  4.89it/s, acc_nat=0.931, acc_rob=0.501]


Epoch 98 [0/383] | loss: 0.0127 (avg: 0.0127) | acc: 1.0000 (avg: 1.0000) |
Epoch 98 [100/383] | loss: 0.0336 (avg: 0.0372) | acc: 1.0000 (avg: 0.9983) |
Epoch 98 [200/383] | loss: 0.0095 (avg: 0.0372) | acc: 1.0000 (avg: 0.9981) |
Epoch 98 [300/383] | loss: 0.0327 (avg: 0.0421) | acc: 1.0000 (avg: 0.9978) |


[eval] 379/1000:  38%|███▊      | 379/1000 [01:15<02:03,  5.05it/s, acc_nat=0.353, acc_rob=0.195]

Epoch 99 [100/383] | loss: 0.0186 (avg: 0.0392) | acc: 1.0000 (avg: 0.9979) |
Epoch 99 [200/383] | loss: 0.0130 (avg: 0.0420) | acc: 1.0000 (avg: 0.9978) |
Epoch 99 [300/383] | loss: 0.0175 (avg: 0.0400) | acc: 1.0000 (avg: 0.9982) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:21<00:00,  4.97it/s, acc_nat=0.923, acc_rob=0.511]
